In [13]:
#treinamento do modelo

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import joblib

# Carregar os dados financeiros de candles a partir de um arquivo CSV
df = base

# Pre-processar os dados
# Eliminar valores ausentes
df.dropna(inplace=True)
# Converter datas em variáveis numéricas
df['DATE'] = pd.to_datetime(df['DATE'].apply(lambda x: datetime.datetime(x.year, x.month, x.day)))
df['DATE'] = df['DATE'].astype('int64') / 10**9
# Calcular o aumento de 10% apenas em relação ao preço de fechamento
df['aumento'] = (df['CLOSE'] / df['CLOSE'].shift(1) - 1) > 0.1
# Normalizar as variáveis
df_norm = (df - df.mean()) / df.std()
# Eliminar a primeira linha, que não tem dados de aumento
df_norm.dropna(inplace=True)

# Dividir os dados em conjuntos de treinamento e teste
X = df_norm.drop('aumento', axis=1)
y = df_norm['aumento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar um modelo de regressão logística
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Salvar o modelo treinado
joblib.dump(lr, 'modelo_treinado.pkl')

# Avaliar o desempenho do modelo
score = lr.score(X_test, y_test)
print('Acurácia:', score)


C:\Users\igorb\AppData\Local\Temp\ipykernel_13716\1382165601.py:20: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_norm = (df - df.mean()) / df.std()
C:\Users\igorb\AppData\Local\Temp\ipykernel_13716\1382165601.py:20: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_norm = (df - df.mean()) / df.std()


UFuncTypeError: ufunc 'subtract' cannot use operands with types dtype('<M8[ns]') and dtype('float64')

In [7]:
# IMPORTAÇÕES

from consumidor import consumer, atualizacao_carteira
from relatorio import analise_tecnica, grafico, noti_telegram
import yfinance as yf
import finplot as fplt
import numpy as np
import pandas as pd
import mysql.connector
import datetime
import pytz
import os 
from dotenv import load_dotenv, find_dotenv


# CONEXÃO COM BANCO DE DADOS
load_dotenv(find_dotenv())

conn = mysql.connector.connect(
  host= os.getenv('host'),
  user= os.getenv('user'),
  password= os.getenv('password'),
  database="bolsa_valores"
)
cursor = conn.cursor()

# ATUALIZAÇÃO DA BASE DE DADOS DE COTAÇÕES

#consumer()

base = pd.read_sql('SELECT * FROM bolsa_valores.cotacao where DATE >= DATE_SUB(CURDATE(), INTERVAL 12 MONTH)', conn)

base.info()

C:\Users\igorb\AppData\Local\Temp\ipykernel_13716\3517861876.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  base = pd.read_sql('SELECT * FROM bolsa_valores.cotacao where DATE >= DATE_SUB(CURDATE(), INTERVAL 12 MONTH)', conn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24348 entries, 0 to 24347
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   DATE       24348 non-null  object 
 1   SYMBOL     24348 non-null  object 
 2   OPEN       24348 non-null  float64
 3   HIGH       24348 non-null  float64
 4   LOW        24348 non-null  float64
 5   CLOSE      24348 non-null  float64
 6   VOLUME     24348 non-null  int64  
 7   DIVIDENDS  24348 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 1.5+ MB


In [ ]:
#utilizar modelo treinado


# Carregar o modelo treinado
lr = joblib.load('modelo_treinado.pkl')


# Pre-processar os dados
# Converter datas em variáveis numéricas
novos_dados['data'] = pd.to_datetime(novos_dados['data']).astype(int) / 10**9


# Prever em quantos dias o preço da ação atingirá um aumento de pelo menos 10%
novos_dados = pd.read_csv('novos_dados_candles.csv')
novos_dados_norm = (novos_dados - df.mean()) / df.std()
proba = lr.predict_proba(novos_dados_norm)
idx = (proba[:, 1] > 0.5).argmax()
dias = idx + 1
print('O preço da ação atingirá um aumento de pelo menos 10% em', dias, 'dias.')


